# Simple Chromatographic Processes

Liquid chromatography is a technique for the separation of mixtures dissolved in a fluid. That fluid is called the mobile phase, which carries the mixture through a structure holding another material, called the stationary phase.

The various components of the mixture interact with different strengths with the stationary phase. Therefore they travel at different speeds, causing them to separate.

Different mechanisms can be used for the separation:
 - adsorption,
 - ion exchange,
 - hydrophobic interactions,
 - size exclusion,
 - etc.

For each mechanism, various stationary phases are available.

There are broadly speaking, _two_ types of chromatography:

__Preparative__ and __analytical__

_Preparative_ targets isolation and purification of "large" quantities of a substance.

_Analytical_ serves to identify or quantify analytes in a mixture.

Generally, chromatographic models are used for optimization of _preparative_ processes.

For modelling these processes, we have to combine all of the techniques we learnt in the previous lessons:
- Configure unit operations models.
- Associate adsorption models with unit operations.
- Generate dynamic inlet profiles.
- Chemical reactions (if required)

## Example 1: Dextran pulse

In this exercise, we will consider the following system:

```{figure} ./resources/system.png
:width: 50%
:align: center
```

Simplify the model by using a large, inert molecule:
- exclude pore access
    - no pore porosity
    - no diffusion parameters
- exclude binding
    - no binding parameters

Only:
the __bed porosity__ and __axial dispersion__ affect the Dextran pulse.

For the column, assume the following parameters which are usually provided by the manufacturer (or can be measured):
- length: $0.1~m$
- diameter: $0.01~m$
- particle radius: $4.5 \cdot 10^{-5}~m$
- flow rate: $10^{-6}/60~m^3 s^{-1}$

Moreover, since Dextran does not penetrate pores, the film diffusion coefficient can be set to $0~m \cdot s^{-1}$.

Finally, bed porosity and axial dispersion need to be specified.
Usually, these parameters will be estimated using an inverse method (see later tutorials).
For now, assume the following values:
- bed porosity: $0.37$
- axial dispersion: $2.0 \cdot 10^{-7}~m^2 \cdot s^{-1}$

For the injection, we need to Introduce two sections.
In the first section, which lasts $50~s$, the concentration of Dextran at the `INLET` is $1.0~mM$, afterwards it is $0.0~mM$.
The flow rate is a constant $1~mL \cdot min^{-1}$.

```{figure} ./resources/dextran_inlet.png
:width: 50%
:align: center
```

## Example 2: Multi Component Langmuir Separation

Now, we will use the same system, but add `Langmuir` model to the column with two components using batch elution chromatography.
This process is often used for the purification of small molecules like amino acids or sugars.

For the Langmuir isotherm, use the following parameters:
- adsorption rate: $[0.02, 0.03]~m^3 mol^{-1} s^{-1}$
- desorption rate: $[1, 1]~s^{-1}$
- binding capacity: $[100, 100]~mM$

additionally, use these transport parameters:

- particle porosity: $0.33$
- film diffusion: $10^{-4}$

This time, use two `Inlet UnitOperations`, a feed `Inlet` with a concentration of $10 ~mM$ for both components and an eluent `Inlet` with a concentration of $0~mM$.

Again, we create two sections to model the injections. This time by turning off the flow rate of the feed `Inlet` and turning on the flow rate of the eluent `Inlet`.

### A Note on Event Dependencies

Often, multiple `Events` happen simulateneously.
Here, for example, when the feed is turned off, the eluent also needs to be switched on.
To eliminate the need to manually change all event times, dependencies can be specified using `add_event_dependency`.

You can see all the dependent events with the `.dependent_events` method

Now simulate and plot the results.

## Example 3: Load wash elute with Steric Mass Action law binding

The [Steric Mass Action model](https://cadet.github.io/master/modelling/binding/steric_mass_action.html#steric-mass-action-model) takes charges of the molecules into account and is, thus, often used in ion-exchange chromatography.
Each component has a characteristic charge $\nu$ that determines the number of available binding sites $\Lambda$ (ionic capacity) used up by a molecule.
Due to the molecule’s shape, some additional binding sites (steric shielding factor $\sigma$) may be shielded from other molecules and are not available for binding.

The model is given by this eqution:

$$\frac{\mathrm{d} q_i}{\mathrm{d} t} = k_{a,i} c_{p,i}\bar{q}_0^{\nu_i} - k_{d,i} q_i c_{p,0}^{\nu_i}$$

where $c_{p,0}$ denotes the mobile phase salt concentration, and

$$\bar{q}_0 = \Lambda - \sum_{j=1}^{N_{\text{comp}} - 1} \left( \nu_j + \sigma_j \right) q_j$$

is the number of available binding sites.

Using the parameter transformation

$$k_{a,i} = \tilde{k}_{a,i} q_{\text{ref}}^{-\nu_i}$$

$$k_{d,i} = \tilde{k}_{d,i} c_{\text{ref}}^{-\nu_i}$$

we obtain the modified model equation:

$$\frac{\mathrm{d} q_i}{\mathrm{d} t} = \tilde{k}_{a,i} c_{p,i} \left(\frac{\bar{q}_0}{q_{\text{ref}}}\right)^{\nu_i} - \tilde{k}_{d,i} q_i \left(\frac{c_{p,0}}{c_{\text{ref}}}\right)^{\nu_i}$$

This transformation serves as a (partial) nondimensionalization of the adsorption and desorption rates.

The basic goal is to have $\left(\frac{\bar{q}_0}{q_{\text{ref}}}\right) \leq 1$ and $\left(\frac{c_{p,0}}{c_{\text{ref}}}\right) \leq 1$

Recommended choices for $c_{\text{ref}}$ are the average or maximum inlet concentration of the mobile phase modifier $c_0$, and for $q_{\text{ref}}$ the ionic capacity $\Lambda$.
Note that setting the reference concentrations to ${1.0}$ each results in the original binding model.

```{note}
From a practical perspective, modern resins have a very high capacity and large proteins can can have a very high charactistic charge.
If the concentration is not normalized, the system is often numerically unstable.
It may run slowly or not at all.
```

In this example, we will look at a typical process for protein purification.

1. First, a protein salt mixture is loaded on the column and binds to the resin.
2. Then, the column is washed with a lower concentrated salt solution.
3. Finally, the protein is eluted by adding a linear salt gradient.

```{figure} ./resources/lwe_inlet.png
:width: 80%
:align: center

```

First, define the `ComponentSystem` and the parameters for the `StericMassAction` model.
As mentioned earlier, consider a reference concentration in the pore for numeric purposes.

Then, we define the system of unit operations.

The protein is loaded for $7500 s$, then there is a wash step, which takes $2000 s$, and the gradient takes another $5500 s$.

Finally, we set the initial conditions of the column.
We assume, that in the beginning of the process, the stationary phase is fully loaded with  salt.

Now, we run the simulation and plot the results. Because the concentration ranges are very different, we use different scales for both components.